In [1]:
%matplotlib inline  
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

Using TensorFlow backend.


In [2]:
x_train = np.load('kmnist-train-imgs.npz')['arr_0']
y_train = np.load('kmnist-train-labels.npz')['arr_0']

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_test = np.load('kmnist-test-imgs.npz')['arr_0']
y_test = np.load('kmnist-test-labels.npz')['arr_0']

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [7]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.6377 - acc: 0.8064 - val_loss: 0.7003 - val_acc: 0.7780
Epoch 2/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.3664 - acc: 0.8880 - val_loss: 0.5861 - val_acc: 0.8133
Epoch 3/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.3004 - acc: 0.9084 - val_loss: 0.5321 - val_acc: 0.8413
Epoch 4/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.2614 - acc: 0.9209 - val_loss: 0.5009 - val_acc: 0.8541
Epoch 5/10
60000/60000 [==============================] - 2s 33us/step - loss: 0.2374 - acc: 0.9272 - val_loss: 0.4822 - val_acc: 0.8611
Epoch 6/10
60000/60000 [==============================] - 2s 30us/step - loss: 0.2174 - acc: 0.9339 - val_loss: 0.4747 - val_acc: 0.8657
Epoch 7/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.2060 - acc: 0.9365 - val_loss: 0.4637 - val_acc

In [9]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 241s 4ms/step - loss: 0.2619 - acc: 0.9191 - val_loss: 0.0578 - val_acc: 0.9805
Epoch 2/10
60000/60000 [==============================] - 195s 3ms/step - loss: 0.0900 - acc: 0.9733 - val_loss: 0.0383 - val_acc: 0.9860
Epoch 3/10
60000/60000 [==============================] - 196s 3ms/step - loss: 0.0680 - acc: 0.9801 - val_loss: 0.0317 - val_acc: 0.9894
Epoch 4/10
60000/60000 [==============================] - 175s 3ms/step - loss: 0.0526 - acc: 0.9844 - val_loss: 0.0313 - val_acc: 0.9894
Epoch 5/10
60000/60000 [==============================] - 304s 5ms/step - loss: 0.0459 - acc: 0.9863 - val_loss: 0.0370 - val_acc: 0.9868
Epoch 6/10
60000/60000 [==============================] - 314s 5ms/step - loss: 0.0417 - acc: 0.9869 - val_loss: 0.0272 - val_acc: 0.9901
Epoch 7/10
60000/60000 [=========================